# C2Q Method 2

v11

In [1]:
import numpy as np
from IPython.display import Markdown as md

# Random data function used in experiments
def random_data(n_qubits, magnitude=False, use_complex_data=False, seed=None):
    if seed:
        np.random.seed(seed)

    n_states = 2**n_qubits
    x = np.random.rand(n_states)

    if use_complex_data:
        x = x + (np.random.rand(n_states) * 2j) - 1j
    else:
        x = x * 255

    mag = np.linalg.norm(x)
    x_in = x / mag

    if magnitude:
        return x_in, mag
    else:
        return x_in
    
def partial_measurement(psi_in, bits_to_remove):
    psi_out = np.zeros_like(psi_in)  # Output vector with same length as psi_in

    # Create bitmask from bits_to_remove
    bitmask = sum(2**x for x in bits_to_remove)
    bitmask = ~bitmask  # Negative mask of bitmask

    # Reduce psi_in while preserving indices
    for i, x in enumerate(psi_in):
        i_out = i & bitmask  # index in psi_out to insert value
        psi_out[i_out] += (
            np.abs(x) ** 2
        )  # Add the square of the value in psi (probability)
    psi_out = np.sqrt(psi_out)  # Take sqrt to return to a "statevector"

    return psi_out

x_in = random_data(n_qubits=2, use_complex_data=False, seed=2)
# x_in = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0]
# x_in = partial_measurement(x_in, [1])

print(x_in)

[0.52779162 0.03138488 0.66539138 0.52697752]


In [2]:
# Format input array properly
x_in = np.array(x_in)

# Calculate number of necessary qubits and states
num_qubits = int(np.ceil(np.log2(len(x_in))))
num_states = np.power(2, num_qubits, dtype=int)

# Pad input array if necessary
if num_states - len(x_in) != 0: 
    print(f'Input length is not power of 2. Padding...')
    x_in = np.pad(x_in, (0, num_states-len(x_in)), 'constant', constant_values=0)
    print(f'New input: {x_in}')

# Normalize input array if necessary
if np.linalg.norm(x_in) != 1:
    print(f'Input is not normalized. Normalizing...')
    x_in = x_in / np.linalg.norm(x_in)
    print(f'New input: {x_in}')

Input is not normalized. Normalizing...
New input: [0.52779162 0.03138488 0.66539138 0.52697752]


In [3]:
### ============================ ###
###  Calculate input parameters  ###
### ============================ ###

# Generates P matrix as in Eq. 17
# Returns tuple of alpha and beta matrices from P, see Eqs. 18 and 19
def get_ab(x_in):
    x_new = np.reshape(x_in, (int(len(x_in)/2), 2))
    i_max = int(len(x_new))
    j_max = int(np.ceil(np.log2(len(x_in))))

    P = np.zeros((i_max, j_max), dtype=complex)
    alpha = np.zeros((i_max, j_max), dtype=complex)
    beta = np.zeros((i_max, j_max), dtype=complex)
    for j in range(j_max):
        
        # TODO: this portion may have optimization potential
        for (i, x) in enumerate(x_new):
            if j == 0:
                p = np.power(np.linalg.norm(x), 2)

                if p == 0:
                    a = 1
                    b = 0
                else:
                    a = x[0] / np.linalg.norm(x)
                    b = x[1] / np.linalg.norm(x)
            elif i >= i_max / (2**j):
                p = 0 
                a = 1
                b = 0
            else:
                p = P[2*i,j-1] + P[2*i+1,j-1]

                if p == 0:
                    a = 1
                    b = 0
                else:  
                    a = np.sqrt(P[2*i,j-1] / p)
                    b = np.sqrt(P[2*i+1,j-1] / p)

            # This is purely done for readability    
            P[i,j] = p
            alpha[i,j] = a
            beta[i,j] = b

    return (alpha, beta)

# Returns values of theta, phi, r, and t according to Eq. 20
def get_params(alpha, beta):
    print(np.abs(alpha))
    print(np.abs(beta))
    
    alpha_mag = np.abs(alpha)
    alpha_phase = np.angle(alpha)
    beta_mag = np.abs(beta)
    beta_phase = np.angle(beta)

    with np.errstate(divide='ignore'):
        theta = 2*np.arctan(beta_mag/alpha_mag)
    phi = beta_phase - alpha_phase
    r = np.sqrt(alpha_mag**2 + beta_mag**2)
    t = beta_phase + alpha_phase

    return theta, phi, r, t

# Returns tuple of theta, phi, r, and t tensors given input data
def input_data(x_in):
    return get_params(*get_ab(x_in))

theta_array, phi_array, r_array, t_array = input_data(x_in)

[[0.99823666 0.        ]
 [0.78392578 0.        ]]
[[0.05935967 0.        ]
 [0.62085455 0.        ]]


/var/folders/sy/jspx4wmd7sd9lqc7clfzx4jw0000gn/T/ipykernel_74241/1232579756.py:65: RuntimeWarning: invalid value encountered in divide
  theta = 2*np.arctan(beta_mag/alpha_mag)


In [4]:
# Basic gates

def Ry(theta):
    return np.matrix(f'{np.cos(theta/2)}, {-np.sin(theta/2)}; {np.sin(theta/2)}, {np.cos(theta/2)}')

def Rz(phi):
    return np.matrix(f'{np.exp(-1j*phi/2)}, {0}; {0}, {np.exp(1j*phi/2)}')

def Uij(theta, phi, r, t):
    #return r * np.exp(1j*t/2) * (Rz(phi) @ Ry(theta))
    return (Rz(phi) @ Ry(theta) @ Rz(-t)) * r
    # return (Rz(phi-t) @ Ry(theta)) * r

In [5]:
# Contruct each Uj in pyramidal structure
Uj_array = []

for j in range(num_qubits):
    n_j = num_qubits - 1 - j
    i_max = 2**(n_j)

    theta_j = theta_array[0:i_max,j]
    phi_j = phi_array[0:i_max,j]
    r_j = r_array[0:i_max,j]
    t_j = t_array[0:i_max,j]
    
    Uj = np.asmatrix(np.zeros((2*i_max, 2*i_max)), dtype=complex)
    for i, (theta, phi, r, t) in enumerate(zip(theta_j, phi_j, r_j, t_j)):
        Uj[2*i:2*i+2, 2*i:2*i+2] = Uij(theta, phi, r, t)

    Uj_array.append(Uj)
    # print(Uj)

ValueError: malformed node or string on line 1: <ast.Name object at 0x112015d80>

In [ ]:
# 1 qubit |0> state
zero_state = np.matrix('1;0')

# Create initial quantum state
psi_0 = zero_state
for _ in range(num_qubits-1):
    psi_0 = np.kron(zero_state, psi_0)

In [ ]:
# Find matrix for entire circuit
U_block = np.eye(2**num_qubits)
for (n_j, U_j) in enumerate(reversed(Uj_array)):
    U_j_kron = np.kron(U_j, np.eye(2**(num_qubits - 1 - n_j)))
    U_block = U_j_kron @ U_block

# Encode state
psi = U_block @ psi_0

In [ ]:
# Helper function for printing matrix in LATEX format
def matrix(M: np.matrix) -> str:
    output = '\\begin{bmatrix}'

    n_cols = 1
    if len(M.shape) == 2:
        _, n_cols = M.shape

    for i, m in enumerate(np.nditer(M)):
        output += f'{m:.03f}'

        if ((i+1) % n_cols) == 0:
            output += '\\\\'
        else:
            output += '&'

    output += '\\end{bmatrix}'

    return output

# Test if matrices are unitary
def is_unitary(M: np.matrix) -> bool:
    M = np.asmatrix(M)

    if len(M.shape) != 2:
        return False;

    n_rows, n_cols = M.shape

    if n_rows != n_cols:
        return False

    M_squared = M @ M.getH()
    M_squared = M_squared.round(3)

    # print(M_squared)

    return np.array_equal(M_squared, np.eye(n_rows))    

In [ ]:
# Print important values in LATEX

md_str  = f'Input data: ${matrix(x_in)}$, '
md_str += f'Number of qubits: {num_qubits}\n'

md_str += '\n'

md_str += f'Matrix of parameters: \n\n'
md_str += f'$\\theta = {matrix(theta_array)}$, '
md_str += f'$\phi = {matrix(phi_array)}$, '
md_str += f'$t = {matrix(t_array)}$, '

# I don't know how to fix r
md_str += f'$r = {matrix(r_array)}$\n'

md_str += '\n'

md_str += f'Each $U_j$ matrix defined in C2Q Method 2: \n\n'
for j, Uj in enumerate(Uj_array):
    md_str += f'$U_{j} = {matrix(Uj)}$\n'
    md_str += f'(Unitary: {is_unitary(Uj)})\n\n'

md_str += '\n'

md_str += f'The matrix for the circuit overall: '
md_str += f'$\left( U^{{C2Q-2}}\lvert 0 \\rangle^{{\otimes n}} = \\vert \Psi \\rangle \\right)$ \n\n'
md_str += f'$U^{{C2Q-2}} = {matrix(U_block)}$\n'
md_str += f'(Unitary: {is_unitary(U_block)})\n\n'

md_str += '\n'

md_str += f'Final encoded state: \n\n'
md_str += f'$\lvert \Psi \\rangle = {matrix(psi)}$\n'

md_str += '\n'

md_str += f'Is final state correct?: {np.allclose(x_in, np.transpose(psi))}\n'

md(md_str)